# BASIC STATISTICS-2

Problem Statement : Hospital Patient Data Analysis
Context:
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.
Tasks:
1.	Load the patient dataset and show summary with info().
2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
4.	Use groupby to find total bill amount per department.
5.	Remove duplicate patient records based on PatientID.
6.	Fill missing BillAmount values with the mean bill amount.
7.	Merge the billing dataset with patient dataset on PatientID.
8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).
Expected Outcome:
•	Final cleaned dataset with accurate billing info.
•	All missing values handled, merged dataset across PatientID.
•	Ability to perform further analytics on department-wise revenue or doctor performance.


In [1]:
import pandas as pd

In [2]:
patient_path = r"C:\Users\91705\OneDrive\Desktop\ExcelR\Assignments\.csv files\Patient_Data.csv"
billing_path = r"C:\Users\91705\OneDrive\Desktop\ExcelR\Assignments\.csv files\Billing_Data.csv"

In [3]:
patient_df = pd.read_csv(patient_path)
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 464.0+ bytes


In [4]:
patient_billing_df = patient_df[['PatientID', 'Department', 'Doctor', 'BillAmount']]
patient_billing_df.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [5]:
cleaned_patient_df = patient_df.drop(columns=['ReceptionistID', 'CheckInTime'], errors='ignore')
cleaned_patient_df.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [6]:
dept_billing = patient_billing_df.groupby('Department')['BillAmount'].sum().reset_index()
dept_billing

,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


In [7]:
unique_patient_df = patient_df.drop_duplicates(subset='PatientID')
unique_patient_df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [8]:
mean_bill = patient_df['BillAmount'].mean()
patient_df['BillAmount'] = patient_df['BillAmount'].fillna(mean_bill)
patient_df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,5925.0,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,5925.0,2,2023-01-14 08:45


In [9]:
billing_df = pd.read_csv(billing_path)
merged_df = pd.merge(patient_df, billing_df, on='PatientID', how='inner')
merged_df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00,2000,3000
1,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00,2000,3000
2,102,Bob,Neurology,Dr. John,5925.0,2,2023-01-11 10:30,1500,3500
3,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00,2500,5000
4,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00,3000,3200


In [10]:
new_patients = pd.DataFrame({
    'PatientID': [2001, 2002],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. A', 'Dr. B'],
    'BillAmount': [5000, 7000]
})

extended_df = pd.concat([merged_df, new_patients], ignore_index=True)
extended_df.tail()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount
3,103,Charlie,Orthopedics,Dr. Lee,7500.0,1.0,2023-01-12 11:00,2500.0,5000.0
4,104,David,Cardiology,Dr. Smith,6200.0,3.0,2023-01-13 12:00,3000.0,3200.0
5,105,Eva,Dermatology,Dr. Rose,5925.0,2.0,2023-01-14 08:45,1000.0,4000.0
6,2001,NaN,Cardiology,Dr. A,5000.0,NaN,NaN,NaN,NaN
7,2002,NaN,Neurology,Dr. B,7000.0,NaN,NaN,NaN,NaN


In [11]:
new_cols = pd.DataFrame({
    'InsuranceCovered': (['Yes', 'No'] * len(extended_df))[:len(extended_df)],
    'FinalAmount': extended_df['BillAmount'] * 0.9
})

final_df = pd.concat([extended_df, new_cols], axis=1)
final_df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,2000.0,3000.0,Yes,4500.0
1,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,2000.0,3000.0,No,4500.0
2,102,Bob,Neurology,Dr. John,5925.0,2.0,2023-01-11 10:30,1500.0,3500.0,Yes,5332.5
3,103,Charlie,Orthopedics,Dr. Lee,7500.0,1.0,2023-01-12 11:00,2500.0,5000.0,No,6750.0
4,104,David,Cardiology,Dr. Smith,6200.0,3.0,2023-01-13 12:00,3000.0,3200.0,Yes,5580.0


In [12]:
final_df.info()
final_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         8 non-null      int64  
 1   Name              6 non-null      object 
 2   Department        8 non-null      object 
 3   Doctor            8 non-null      object 
 4   BillAmount        8 non-null      float64
 5   ReceptionistID    6 non-null      float64
 6   CheckInTime       6 non-null      object 
 7   InsuranceCovered  6 non-null      float64
 8   FinalAmount       6 non-null      float64
 9   InsuranceCovered  8 non-null      object 
 10  FinalAmount       8 non-null      float64
dtypes: float64(5), int64(1), object(5)
memory usage: 832.0+ bytes


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,2000.0,3000.0,Yes,4500.0
1,101,Alice,Cardiology,Dr. Smith,5000.0,1.0,2023-01-10 09:00,2000.0,3000.0,No,4500.0
2,102,Bob,Neurology,Dr. John,5925.0,2.0,2023-01-11 10:30,1500.0,3500.0,Yes,5332.5
3,103,Charlie,Orthopedics,Dr. Lee,7500.0,1.0,2023-01-12 11:00,2500.0,5000.0,No,6750.0
4,104,David,Cardiology,Dr. Smith,6200.0,3.0,2023-01-13 12:00,3000.0,3200.0,Yes,5580.0
5,105,Eva,Dermatology,Dr. Rose,5925.0,2.0,2023-01-14 08:45,1000.0,4000.0,No,5332.5
6,2001,NaN,Cardiology,Dr. A,5000.0,NaN,NaN,NaN,NaN,Yes,4500.0
7,2002,NaN,Neurology,Dr. B,7000.0,NaN,NaN,NaN,NaN,No,6300.0
